# Unit tests con TensorFlow

In [1]:
import sys
import numpy as np
import tensorflow as tf

In [2]:
session = tf.Session()

In [3]:
data_dir = "../../datasets/MNIST_data/"
mnist = tf.keras.datasets.mnist
(train_xdata, train_labels), (test_xdata, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 22s 2us/step


In [5]:
train_xdata = train_xdata/255.0
test_xdata = test_xdata/255.0

In [10]:
batch_size = 100
learning_rate = 0.005
evaluation_size = 100
image_width = train_xdata[0].shape[1]
image_height = train_xdata[0].shape[0]
target_size = max(train_labels) + 1
num_channels = 1
generations = 100
eval_every = 5
conv1_features = 25
conv2_features = 50
max_pool_size1 = 2
max_pool_size2 = 2
fully_connected_size1 = 100
dropout_prob = 0.75

In [13]:
x_input_shape = (batch_size, image_width, image_height, num_channels)
x_input = tf.placeholder(shape=x_input_shape, dtype=tf.float32)
y_target = tf.placeholder(shape=(batch_size), dtype=tf.int32)

eval_input_shape = (evaluation_size, image_width, image_height, num_channels)
eval_input = tf.placeholder(shape=eval_input_shape, dtype=tf.float32)
eval_target = tf.placeholder(shape=(evaluation_size), dtype=tf.float32)

dropout = tf.placeholder(dtype=tf.float32, shape=())

In [14]:
conv1_weight = tf.Variable(tf.truncated_normal(shape=[4,4,num_channels,conv1_features], stddev=0.1, 
                                               dtype=tf.float32))
conv1_bias = tf.Variable(tf.zeros(shape=[conv1_features], dtype=tf.float32))

conv2_weight = tf.Variable(tf.truncated_normal(shape=[4,4,conv1_features,conv2_features], stddev=0.1, 
                                               dtype=tf.float32))
conv2_bias = tf.Variable(tf.zeros(shape=[conv2_features], dtype=tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
result_width = image_width//(max_pool_size1+max_pool_size2)
result_height = image_height//(max_pool_size1+max_pool_size2)

full1_input_size = result_height*result_width*conv1_features
full1_weight = tf.Variable(tf.truncated_normal(shape=[full1_input_size, fully_connected_size1], stddev=0.1, 
                                               dtype=tf.float32))
full1_bias = tf.Variable(tf.zeros(shape=[fully_connected_size1], dtype=tf.float32))

output_weigh = tf.Variable(tf.truncated_normal(shape=[fully_connected_size1, target_size], stddev=0.1, 
                                               dtype=tf.float32))
output_bias = tf.Variable(tf.zeros(shape=[target_size]))

In [21]:
def cnn(input_data):
    conv1 = tf.nn.conv2d(input=input_data, filter=conv1_weight, strides=[1,1,1,1], padding="SAME")
    relu1 = tf.nn.relu(tf.nn.bias_add(value=conv1, bias=conv1_bias))
    maxpool1 = tf.nn.max_pool(value=relu1, ksize=[1,max_pool_size1,max_pool_size1,1], 
                              strides=[1,max_pool_size1,max_pool_size1,1], padding="SAME")
    
    conv2 = tf.nn.conv2d(input=maxpool1, filter=conv2_weight, strides=[1,1,1,1], padding="SAME")
    relu2 = tf.nn.relu(tf.nn.bias_add(value=conv2, bias=conv2_bias))
    maxpool2 = tf.nn.max_pool(value=relu2, ksize=[1,max_pool_size2,max_pool_size2,1], 
                              strides=[1,max_pool_size2,max_pool_size2,1], padding="SAME")
    
    final_conv_shape = maxpool2.get_shape().as_list()
    final_shape = final_conv_shape[1] * final_conv_shape[2] * final_conv_shape[3]
    flat_output = tf.reshape(maxpool2, [final_conv_shape[0], final_shape])
    
    fully = tf.nn.relu(tf.add(tf.matmul(flat_output, full1_weight), full1_bias))
    output = tf.add(tf.matmul(fully, output_weigh), output_bias)
    
    return tf.nn.dropout(output, keep_prob=dropout_prob)